# CS Chatbot LLM – Process Overview

This notebook captures the end-to-end flow of the chat demo so stakeholders can understand how intake, processing, and delivery connect. It mirrors the code paths in the repo without sending any live messages.

## Architecture Snapshot
- **Webhook/API layer** (`POST /chat/enqueue`) receives chat payloads from a widget or integration and writes them to the Excel queue.
- **Queue store** (`data/email_queue.xlsx`) holds inbound/outbound turns plus metadata (`status`, `delivery_status`, `response_payload`).
- **Chat worker** (`tools/chat_worker.py`) claims queued items, runs `ChatService` (knowledge lookup + LLM fallback), and records replies/decisions.
- **Dispatcher** (`tools/chat_dispatcher.py`) acknowledges processed turns and writes the demo transcript to `data/chat_web_transcript.jsonl`.
- **Visual surfaces** (Streamlit dashboard and static HTML demo) read the queue/transcript to illustrate the conversation.

### Sequence Diagram (textual)
```
Teams/web widget -> FastAPI webhook (/chat/enqueue)
                 -> Excel queue (status=queued)
Chat worker      -> claim row -> ChatService.respond -> update row (status=responded/handoff)
Dispatcher       -> filter responded rows -> delivery adapter (Teams/web demo)
                 -> transcript JSONL (for demos and auditing)
```

## Knowledge Examples
The worker still relies on the legacy FAQ/knowledge table. The cell below previews the canonical facts that the LLM and guardrails ground against.

In [20]:
from pathlib import Path
import pandas as pd

template = Path('..') / 'docs' / 'customer_service_template.md'
rows = []
with template.open('r', encoding='utf-8') as fh:
    for line in fh:
        raw = line.strip()
        if not raw or raw.startswith('#'):
            continue
        if raw.startswith('|'):
            cells = [col.strip() for col in raw.strip('|').split('|')]
            rows.append(cells)

# remove separator rows and empty entries
rows = [r for r in rows if '-' not in ''.join(r)]
header, *data_rows = rows
df = pd.DataFrame(data_rows, columns=[col for col in header if col])
df = df.rename(columns=lambda c: c.strip())
df[['Key', 'Value']].head(10)


,Key,Value
0,company_name,Aurora Gadgets
1,founded_year,1990
2,warranty_policy,Our warranty policy covers every Aurora device...
3,return_policy,Customers may return unused products within 30...
4,shipping_time,Orders ship worldwide and arrive within 5–7 bu...
5,loyalty_program,Aurora Rewards grants points on every purchase...
6,support_email,support@auroragadgets.example


## Sending a Chat Message (demo)
```bash
# enqueue via FastAPI
curl -X POST http://localhost:8000/chat/enqueue      -H 'Content-Type: application/json'      -d '{"conversation_id":"teams-demo-1","text":"Can you summarise my loyalty benefits?","end_user_handle":"teams-user"}'

# run worker + dispatcher (from shell)
python tools/chat_worker.py --queue data/email_queue.xlsx --processor-id teams-worker
python tools/chat_dispatcher.py --queue data/email_queue.xlsx --dispatcher-id teams-dispatch --adapter web-demo
```
In production the dispatcher adapter would call the Teams webhook instead of logging to JSONL.

In [21]:
# Example payload the Teams adapter would POST
teams_message = {
    'type': 'message',
    'text': 'Aurora Rewards thanks you for your loyalty!',
    'conversation': {'id': 'teams-demo-1'},
    'channelId': 'msteams',
}
teams_message

{'type': 'message',
 'text': 'Aurora Rewards thanks you for your loyalty!',
 'conversation': {'id': 'teams-demo-1'},
 'channelId': 'msteams'}

## Benchmark Snapshot
Use `python tools/benchmark_chat.py --queue data/benchmark_queue.xlsx --reset --messages-json data/bench_messages.json --repeat 1 --dispatch`
to capture latency and replies. Free-form prompts route through the LLM; FAQ-style prompts hit the knowledge base instantly. Results include the generated reply text for documentation.

## Interactive Demo (run inside this notebook)
Execute the cell below to enqueue a few sample messages, run the chat worker, dispatch the replies, and inspect the queue.

> Tip: Set `MODEL_BACKEND="stub"` for fast deterministic output or keep `MODEL_BACKEND="ollama"` if the Ollama server is running.

In [25]:
from pathlib import Path
import json
import pandas as pd
import sys

ROOT = Path('..').resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from tools import chat_ingest, chat_worker, chat_dispatcher
from app.chat_service import ChatService

QUEUE_PATH = ROOT / 'data' / 'notebook_demo_queue.xlsx'
TRANSCRIPT_PATH = ROOT / 'data' / 'notebook_demo_transcript.jsonl'

messages = [
    {
        "conversation_id": "notebook-1",
        "text": "Hi there! What products does Aurora Gadgets sell?",
        "end_user_handle": "demo-user",
    },
    {
        "conversation_id": "notebook-1",
        "text": "When were you founded?",
    },
    {
        "conversation_id": "notebook-2",
        "text": "Can you explain the loyalty program benefits in two sentences?",
    },
]

if QUEUE_PATH.exists():
    QUEUE_PATH.unlink()
if TRANSCRIPT_PATH.exists():
    TRANSCRIPT_PATH.unlink()

inserted = chat_ingest.ingest_messages(QUEUE_PATH, messages)
service = ChatService()
processed = 0
while chat_worker.process_once(QUEUE_PATH, processor_id='notebook-worker', chat_service=service):
    processed += 1

chat_dispatcher.dispatch_once(
    QUEUE_PATH,
    dispatcher_id='notebook-dispatcher',
    adapter='web-demo',
    adapter_target=str(TRANSCRIPT_PATH),
)

print(f'Inserted {inserted} message(s); processed {processed}.')
queue_df = pd.read_excel(QUEUE_PATH)
queue_df[['conversation_id', 'payload', 'status', 'response_payload', 'delivery_status']]


Processed conversation notebook-1 -> status=responded latency=0.000s
Processed conversation notebook-1 -> status=responded latency=0.000s
Processed conversation notebook-2 -> status=responded latency=0.000s
Dispatched 3 chat message(s) -> status=delivered
Inserted 3 message(s); processed 3.


,conversation_id,payload,status,response_payload,delivery_status
0,notebook-1,Hi there! What products does Aurora Gadgets sell?,delivered,"{""type"": ""text"", ""content"": ""We are Aurora Gad...",sent
1,notebook-1,When were you founded?,delivered,"{""type"": ""text"", ""content"": ""Aurora Gadgets wa...",sent
2,notebook-2,Can you explain the loyalty program benefits i...,delivered,"{""type"": ""text"", ""content"": ""Aurora Rewards gr...",sent


In [26]:
from pprint import pprint

if TRANSCRIPT_PATH.exists():
    entries = [
        json.loads(line)
        for line in TRANSCRIPT_PATH.read_text(encoding='utf-8').splitlines()
        if line.strip()
    ]
    pprint(entries[-len(messages):])
else:
    print('No transcript logged yet.')

[{'channel': 'web_chat',
  'conversation_id': 'notebook-1',
  'delivery_route': 'web-demo',
  'end_user_handle': 'demo-user',
  'message_id': 'a67f7d47-54f7-4917-bc65-b9f082180b9f',
  'response': {'content': 'We are Aurora Gadgets, and we are happy to help.',
               'decision': 'answer',
               'matched_fact': 'company_name',
               'type': 'text'},
  'timestamp': '2025-09-29T09:22:14.059745+00:00'},
 {'channel': 'web_chat',
  'conversation_id': 'notebook-1',
  'delivery_route': 'web-demo',
  'end_user_handle': 'demo-user',
  'message_id': '56337a20-bb23-42f1-916e-57223069b28e',
  'response': {'content': 'Aurora Gadgets was founded in 1990.',
               'decision': 'answer',
               'matched_fact': 'founded_year',
               'type': 'text'},
  'timestamp': '2025-09-29T09:22:14.060746+00:00'},
 {'channel': 'web_chat',
  'conversation_id': 'notebook-2',
  'delivery_route': 'web-demo',
  'end_user_handle': 'demo-user',
  'message_id': '5f4a220e-6801-

## Transcript Summary
Below we show the last few replies captured by the notebook demo (left) and the main dispatcher log (right).

In [27]:
from pathlib import Path
import json
import pandas as pd

notebook_transcript = Path('..') / 'data' / 'notebook_demo_transcript.jsonl'
dispatcher_transcript = Path('..') / 'data' / 'chat_web_transcript.jsonl'

def load_entries(path: Path):
    if not path.exists():
        return []
    entries = [
        json.loads(line)
        for line in path.read_text(encoding='utf-8').splitlines()
        if line.strip()
    ]
    return entries[-5:]

def format_entries(entries):
    records = []
    for entry in entries:
        response = entry.get('response', {})
        records.append({
            'time': entry.get('timestamp', '')[:19],
            'conversation': entry.get('conversation_id'),
            'decision': response.get('decision'),
            'reply': response.get('content'),
        })
    return pd.DataFrame(records)

demo_df = format_entries(load_entries(notebook_transcript))
log_df = format_entries(load_entries(dispatcher_transcript))

{'notebook_demo': demo_df, 'dispatcher_log': log_df}

{'notebook_demo':                   time conversation decision  \
 0  2025-09-29T09:22:14   notebook-1   answer   
 1  2025-09-29T09:22:14   notebook-1   answer   
 2  2025-09-29T09:22:14   notebook-2   answer   
 
                                                reply  
 0   We are Aurora Gadgets, and we are happy to help.  
 1                Aurora Gadgets was founded in 1990.  
 2  Aurora Rewards grants points on every purchase...  ,
 'dispatcher_log':                   time  conversation decision  \
 0  2025-09-29T07:56:43       bench-1   answer   
 1  2025-09-29T07:56:43       bench-2   answer   
 2  2025-09-29T08:04:19  bench-long-2   answer   
 3  2025-09-29T08:10:07  bench-long-2   answer   
 4  2025-09-29T08:10:47  bench-long-2   answer   
 
                                                reply  
 0  Our headquarters is located in Helsinki, Finland.  
 1  Aurora Rewards grants points on every purchase...  
 2  Aurora Rewards grants points on every purchase...  
 3  Aurora Rewar